In [91]:
import os
import sys

In [92]:
module_path = os.path.abspath(os.path.join('../src'))
if module_path not in sys.path:
    sys.path.append(module_path)

from Classes import Customer, ProfileBase
from ExperimentUtils import sanity_check_purchase_upload_events, recommendations_to_csv,\
        run_personalized_recommendation_experiment
from TransactionsUtils import TransactionsHandler
from FeatureUtils import read_numpy_features_matrix

In [93]:
os.environ['CUDA_VISIBLE_DEVICES'] = '1'

In [94]:
tmp = read_numpy_features_matrix('./curatornet_10m/', 'item_vectors.npy', 'ids')
item_embeddings = tmp['features']
id2index = tmp['id2index']
index2id = tmp['index2id']

In [95]:
item_embeddings.shape

(13297, 200)

In [96]:
ids_with_features = set(index2id)
len(ids_with_features)

13297

In [97]:
artworks_dict = TransactionsHandler.artworks_dict

In [98]:
customers_dict = { cid : Customer() for cid in TransactionsHandler.sales_df.user_id_hash.unique() }

In [99]:
# ---- upload events -----
upload_events = TransactionsHandler.upload_events

# ---- purchase events -----
purchase_session_events = TransactionsHandler.purchase_session_events

# distribute purchases among customers
for pe in purchase_session_events:
    customers_dict[pe.customer_id].append_purchase_session(pe)

# --- join events and sort by timestamp ----
time_events = upload_events + purchase_session_events
time_events.sort(key=lambda x : x.timestamp)

In [100]:
print("len(upload_events) = ", len(upload_events))
print("len(purchase_session_events) = ", len(purchase_session_events))
print("len(time_events) = ", len(time_events))

len(upload_events) =  7742
len(purchase_session_events) =  4897
len(time_events) =  12639


In [101]:
sanity_check_purchase_upload_events(time_events, artworks_dict)

CHECK: event types are correct
CHECK: events ordered by timestamp
CHECK: products are only uploaded once
CHECK: products can only be purchased if present in inventory


In [102]:
REC_SIZE = 10
MAX_PROFILE_SIZES = [None]
MAX_PROFILE_TAGS = ['_(maxprofsize=oo)' if size is None else ('_(maxprofsize=%d)' % size)
                                                                for size in MAX_PROFILE_SIZES]

In [103]:
MAX_PROFILE_SIZES, MAX_PROFILE_TAGS

([None], ['_(maxprofsize=oo)'])

In [104]:
class CuratorNetProfile(ProfileBase):
    # --- global -----        
    @classmethod
    def global_purchase_session_event_handler(cls, purch_sess):
        pass

    # --- instance ----    
    def __init__(self, maxprofsize, artworks_dict, network, sess):
        ProfileBase.__init__(self, maxprofsize, artworks_dict)
        self._network = network
        self._sess = sess
    def ready(self):
        return len(self.consumed_artworks) > 0    
    def handle_artwork_added(self, artwork):
        pass        
    def handle_artwork_removed(self, artwork):
        pass
    def rank_inventory_ids(self, inventory_artworks):
        profile_indexes = [id2index[a.id] for a in self.consumed_artworks]
        inventory_indexes = [id2index[a.id] for a in inventory_artworks]
        match_scores = self._network.get_match_scores(self._sess,
            item_embeddings, profile_indexes, inventory_indexes)
        pairs = [(s,i) for s,i in zip(match_scores, inventory_indexes)]
        pairs.sort(reverse=True)
        return [index2id[p[1]] for p in pairs]

In [105]:
import tensorflow as tf
from Networks import CuratorNet_Evaluation

In [106]:
def run_experiments(artworks_dict, customers_dict, time_events, checkpoint_path, version, version_kwargs, version_alias=None):
    if version_alias is None:
        version_alias = version
    with tf.Graph().as_default():
        network = CuratorNet_Evaluation(**version_kwargs)
        gpu_options = tf.GPUOptions(
            per_process_gpu_memory_fraction=0.99,
            allow_growth=True
        )
        config = tf.ConfigProto(gpu_options=gpu_options)
        with tf.Session(config=config) as sess:
            saver = tf.train.Saver()
            saver.restore(sess, tf.train.latest_checkpoint(checkpoint_path))
            for maxprofsize, maxproftag in zip(MAX_PROFILE_SIZES, MAX_PROFILE_TAGS):
                create_profile_func = lambda _: CuratorNetProfile(maxprofsize, artworks_dict, network, sess)
                recommendations = run_personalized_recommendation_experiment(
                    artworks_dict, customers_dict, time_events, create_profile_func, rec_size=REC_SIZE)
                recommendations_to_csv(
                    recommendations,
                    ("../results/@{}{}_{}.csv").format(
                        REC_SIZE, maxproftag, version_alias))

In [107]:
version = 'curatornet_resnet_10m'
version

'curatornet_resnet_10m'

In [108]:
run_experiments(artworks_dict, customers_dict, time_events,
                checkpoint_path='./curatornet_10m/',
                version=version,
                version_kwargs=dict(
                    user_layer_units=[300,300,200],
                    latent_space_dim=200,
                    profile_pooling_mode='AVG+MAX',
                ))

INFO:tensorflow:Restoring parameters from C:\Users\victor_accete\CuratorNet\experiments\curatornet_10m\
---------- starting experiment ------------
500 tests done! elapsed time: 3.17 seconds
1000 tests done! elapsed time: 6.22 seconds
1500 tests done! elapsed time: 9.68 seconds
1978 tests done! elapsed time: 13.15 seconds
** recommendations successfully saved to ../results/@10_(maxprofsize=oo)_curatornet_resnet_10m.csv
